In [43]:
import subprocess
import json
import os
from bit import PrivateKeyTestnet
import web3
from web3 import Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account
from dotenv import load_dotenv
load_dotenv()

from constants import *
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:7545"))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [44]:
from constants import *
mnemonic = os.getenv('MNEMONIC')

In [85]:
def derive_wallets(coin):
    mnemonic = os.getenv('MNEMONIC')

    #ln does not work for me, replaced with actual php instead
    command = f'php hd-wallet-derive\hd-wallet-derive.php -g --mnemonic="{mnemonic}" --cols=path,address,privkey,pubkey --format=json --coin={coin}'

    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()

    keys = json.loads(output)
    return keys

def priv_key_to_account(coin, priv_key):
    if (coin == 'eth'):
        return web3.eth.Account.privateKeyToAccount(priv_key)
    elif (coin == 'btc-test'):
        return PrivateKeyTestnet(priv_key)
    else:
        return None

def create_tx(coin , account, to, amount):
    if (coin == 'eth'):
        return {
            'to': to,
            'from': account.address,
            'value': amount,
            'gas': w3.eth.estimateGas({
                "from": account.address,
                "to": to,
                "value": amount
            }),
            'gasPrice': w3.eth.gasPrice,
            'nonce': w3.eth.getTransactionCount(account.address)
        }
    elif (coin == 'btc-test'):
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])
    else:
        return None

In [55]:
coin_list = [BTC, ETH, BTCTEST]
coins = {}
for coin in coin_list:
    coins[coin] = derive_wallets(coin)

In [89]:
coins[BTCTEST]

[{'path': "m/44'/1'/0'/0/0",
  'address': 'mrEpvqTHeU52Aj5SRsux33V4xwqEmWcbAW',
  'privkey': 'cTvT1LAHD9X3BLiKRJbrtCkErMeRaJwpDrNtZCAQw9MGWycJ2pXf',
  'pubkey': '02c15d8c1d85ca5db116fc6ad7e8c48c3a2be7b3f9877d02069322a52b654dfff7'},
 {'path': "m/44'/1'/0'/0/1",
  'address': 'mpJbDpsFBtu8wDJDsLuHBcGbnf3T2BhEQL',
  'privkey': 'cQbwVLXcXjH75vAnTr4CjFriqmn7yXPzuCxvvMeFjJ91ywES8rM1',
  'pubkey': '03fc68ca7877ee3caeb3febcb43b1d563f39fbbbab333235a758cf9389d43a905c'},
 {'path': "m/44'/1'/0'/0/2",
  'address': 'miRZq1aZpGM4aztZdAPezmUEtNP8f6fwQD',
  'privkey': 'cVt5KTfpsrnKUiMYeqeVequYr5jXby5xnoToMtPzky84rk54UHxR',
  'pubkey': '03a46b7afa200f696edd20e1f93a0cd0ad9b1f484f8062ef0bc5bb420404c1f6dc'},
 {'path': "m/44'/1'/0'/0/3",
  'address': 'mrnKe5nJHmZjcJURvHoa5ptiPTAuRoTzwg',
  'privkey': 'cT6R7RN89QT13LZKVagCD6uz2Y8ZBoe18m8E3HkB9dFA1LFSShCc',
  'pubkey': '030cf5cf0885a4f67688d2943ab996b24d5f30e0bc31de25ff5f969edac8f170b9'},
 {'path': "m/44'/1'/0'/0/4",
  'address': 'msQ2is4R3wNuHpA2AVQNYgK79Kiaf

In [123]:
raw_tx = create_tx(BTCTEST, priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey']), coins[BTCTEST][1]['address'], 0.0000001)
# raw_tx = create_tx(ETH, priv_key_to_account(ETH, coins[ETH][0]['privkey']), coins[ETH][1]['address'], 1)
raw_tx
# dir(w3.eth)

'{"unspents":[{"amount":10000,"confirmations":135,"script":"76a914759c945d8ddf56b911998ed6dd57d4ffea797f0388ac","txid":"789a5e4ffbc51fd4b3c56d4db32acbe430e340483fb0c510759fe3bfd2142e1b","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":94000,"confirmations":5,"script":"76a914759c945d8ddf56b911998ed6dd57d4ffea797f0388ac","txid":"27475ccb59eb4ed22cb68ba3191af4baf02f64eb9dc821831d5f761747bcef57","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":1743775,"confirmations":4,"script":"76a914759c945d8ddf56b911998ed6dd57d4ffea797f0388ac","txid":"6ec886a304aebcac5ebd87b0709e6eec3b367e5aee9c47c681e690ce24603841","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mpJbDpsFBtu8wDJDsLuHBcGbnf3T2BhEQL",10],["mrEpvqTHeU52Aj5SRsux33V4xwqEmWcbAW",1669241]]}'

In [125]:
from bit.network import NetworkAPI

In [124]:
account = priv_key_to_account(BTCTEST, coins[BTCTEST][0]['privkey'])
# signed = account.sign_transaction(raw_tx)
# account.sign_transaction(raw_tx)
account.sign_transaction(raw_tx)

'01000000031b2e14d2bfe39f7510c5b03f4840e330e4cb2ab34d6dc5b3d41fc5fb4f5e9a78010000006a47304402203e4598cf4e526806f9b2e5cc3f3a9b1dad5f494b94f781e21d8203842d4bc67f02200ab4e67171f9577d83b37038f8c314ec9b5e0c928225d3f282b04916a9ca9906012102c15d8c1d85ca5db116fc6ad7e8c48c3a2be7b3f9877d02069322a52b654dfff7ffffffff57efbc4717765f1d8321c89deb642ff0baf41a19a38bb62cd24eeb59cb5c4727000000006b483045022100ce3eb601c18a75d4050632ced121c6ef28a7391b2f4f0a301ed3d43496b73138022018e2f416465d9a122da603739c0a7c56d6c5064dd8070aedd3b67340ea82e60f012102c15d8c1d85ca5db116fc6ad7e8c48c3a2be7b3f9877d02069322a52b654dfff7ffffffff41386024ce90e681c6479cee5a7e363bec6e9e70b087bd5eacbcae04a386c86e000000006a47304402207da452878b70dabb2ddd1ea599d20986c6a89760849c6bb751d4a5afffeee9080220620376b21e5c0912ac679e4d4cb290bab6fb36dbd00397427691bed2e41c6406012102c15d8c1d85ca5db116fc6ad7e8c48c3a2be7b3f9877d02069322a52b654dfff7ffffffff020a000000000000001976a91460628cbc1e2cb0d8fd74571264c6cea0bc68464e88ac79781900000000001976a914759c945d8dd

In [103]:
w3.eth.sendRawTransaction(signed.rawTransaction)

HexBytes('0x46a1b3a3fb18c81211fd074b6edcde722c67a5a9657e854ec391a9e73c0f4e59')